In [7]:
import torch
from CNN.models import MlpRegBaseline
from CNN.datasets import Class_Seq_Dataset
from CNN.datasets import load_class_seq_data
import matplotlib.pylab as plt
import sys

In [8]:
# useful functions:

def load_model(model_path, plot=False):
    cp = torch.load(model_path)
    xval_data = cp["models_data"]
    a = cp["arguments"]

    train_losses = torch.tensor([data["train_losses"] for data in xval_data])
    validation_losses = torch.tensor([data["validation_losses"] for data in xval_data])
    train_losses = torch.permute(train_losses, (1,0))
    validation_losses = torch.permute(validation_losses, (1,0))
    if plot:
        #plot mean validation and loss
        plt.figure(figsize=(9,6))
        plt.plot(range(a.epochs), train_losses.mean(dim=1))
        plt.plot(range(a.epochs), validation_losses.mean(dim=1))
        plt.title("Valid")
        plt.xlabel("Iterations over dataset")
        plt.ylabel("Accuracy")
        plt.title("Mean learning curve of 10 models")
        plt.legend(["Training", "Validation"])
        plt.show()

        #plot the actual validation and loss
        plt.figure(figsize=(9,6))
        plt.plot(range(a.epochs), train_losses)
        plt.plot(range(a.epochs), validation_losses)
        plt.title("Validation and losses")
        plt.xlabel("Iterations over dataset")
        plt.show()
    return xval_data, a

def get_rates(xval_data, a):
    csv_peptides, csv_ba_values, groups = load_class_seq_data("../../data/external/processed/BA_pMHCI.csv", a.threshold)
    dataset = Class_Seq_Dataset(csv_peptides, csv_ba_values, a.encoder)
    accuracies = []
    tpr = []
    fpr = []
    tnr = []
    fnr = []
    nr = []
    pr = []
    for data in xval_data:
        # load the model and data
        model = MlpRegBaseline(neurons_per_layer=a.neurons) 
        model.load_state_dict(data["model"])
        model.eval()

        test_X = dataset.peptides[data["test_indices"]]
        test_label = dataset.labels[data["test_indices"]]

        ### MAKE THE PREDICTION ON THE TEST DATASET ###
        with torch.no_grad():
            test_pred_label = torch.reshape(model(test_X), (-1,))

        ### CALCULATE SENSITIVITY, SPECIFICITY AND OTHER METRICS ###
        # Calculate the confusion tensor (AKA matching matrix) by dividing predictions
        # with true values.
        # To assess performances, first calculate absolute values and adjust it to 
        # the total number of expected postives, negatives:

        # calculate absolute values:
        confusion = test_pred_label.round()/test_label # absolute values for metrics
        tot = test_label.shape[0] # total number of prediction
        pos = float((test_label == 1.).sum()) # total number of positives (truth 1)
        neg = float((test_label == 0.).sum()) # total number of negatives (truth 0)

        # calculate rates:
        tpr.append( float((confusion == 1.).sum()/pos) ) # true positive rate = prediction 1/truth 1
        fnr.append( float((confusion == 0.).sum()/pos)) # false negative rate = predicted 0/truth 1
        tnr.append( float(torch.isnan(confusion).sum()/neg) ) # true negative rate = predicted 0/truth 0
        fpr.append( float((confusion == float("inf")).sum()/neg) ) # false positive rate = predicted 1/truth 0

        # calculate raw accuracy:
        accuracy = float((test_pred_label.round() == test_label).sum()/test_label.shape[0])
        accuracies.append(accuracy)

        # to keep track of truths in test sets
        nr.append( neg/tot ) 
        pr.append( pos/tot )

    ### STORE METRICS ###
    accuracies = torch.tensor(accuracies)
    tpr = torch.tensor(tpr)
    fnr = torch.tensor(fnr)
    tnr = torch.tensor(tnr)
    fpr = torch.tensor(fpr)
    nr = torch.tensor(nr)
    pr = torch.tensor(pr)
    return tpr,fnr,tnr,fpr,nr,pr

In [15]:
models_path = "/home/daqop/Desktop/M2/Internship/python/trained_models/"
# models_path = "/home/daqop/mountpoint_snellius/3D-Vac/src/5_train_models/CNN/I/reg/seq/trained_models/"
# models_path = "/home/daqop/train_branch_3D_Vac/src/5_train_models/CNN/I/classification/seq/trained_models/"
model_names = {
    "shuffled": "mlp_class_baseline_200_neurons_20000_epochs_shuffled.pt",
    "clustered": "mlp_class_baseline_200_neurons_20000_epochs_shuffled.pt",
}
shuffled_xval, shuffled_a = load_model(models_path + model_names["shuffled"])
clustered_xval, clustered_a = load_model(models_path + model_names["clustered"])
s_tpr, s_fnr, s_tnr, s_fpr, s_nr, s_pr = get_rates(shuffled_xval, shuffled_a)
c_tpr, c_fnr, c_tnr, c_fpr, c_nr, c_pr = get_rates(clustered_xval, clustered_a)

mean accuracy: 0.7754361629486084
mean accuracy standard deviation: 0.01158651988953352
mean true positive rate: 0.691307544708252
mean false negative rate: 0.30869239568710327
mean true negative rate: 0.8448594808578491
mean false positive rate: 0.1551404893398285
mean negatives rate in the test set: 0.5454359650611877
mean positives rate in the test set: 0.45456403493881226
